Initial 200 ns simulations

Parameters: 4 fs timestep, no cutoff as the non-bonded method, 1.5 amu hydrogen mass repartitioning and mixed computation precision

In [ ]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout
import time

start_time = time.time()

prmtop = AmberPrmtopFile('input.prmtop')
inpcrd = AmberInpcrdFile('input.inpcrd')

system = prmtop.createSystem(nonbondedMethod=NoCutoff,
        constraints=HBonds, hydrogenMass=1.5*amu, implicitSolvent=GBn2)

integrator = LangevinMiddleIntegrator(298.15*kelvin, 1/picosecond, 0.004*picoseconds)

platform = Platform.getPlatformByName('CUDA')

properties = {'Precision': 'mixed'}

simulation = Simulation(prmtop.topology, system, integrator, platform, properties)
simulation.context.setPositions(inpcrd.positions)
simulation.minimizeEnergy()
simulation.reporters.append(DCDReporter('output.dcd', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True))
simulation.step(50000000)

print ("simulation time:", time.time() - start_time, "s")